## Домашняя работа №3

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.



In [1]:
import pandas as pd
from tqdm import tqdm
import nltk

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#!conda install spacy

In [4]:
import spacy 
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 1000000

In [5]:
def ner_by_spacy(texts):

    entities = []
    labels = []
    for text in tqdm(texts):
        doc = nlp(text)
        for ent in doc.ents:
            entities.append(ent.text)
            labels.append(ent.label_)

    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [6]:
def ner_by_nltk(texts):
    
    entities = []
    labels = []
    
    for text in tqdm(texts):
        words = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(words)
        chunks = nltk.ne_chunk(pos_tags, binary=False)

        for chunk in chunks:
            if hasattr(chunk, 'label'):
                entities.append(' '.join(c[0] for c in chunk))
                labels.append(chunk.label())
    
    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [7]:
test_article = "The decorated ceilings of the Natural History Museum in South Kensington, London, \
    were designed by the museum's architect Alfred Waterhouse, and were unveiled at the building's \
    opening in 1881. The ceiling of the large Central Hall (pictured) consists of 162 panels, 108 of \
    which depict plants considered significant to the history of the museum, to the British Empire or \
    to the museum's visitors. The remaining 54 are highly stylised decorative botanical paintings. The \
    ceiling of the smaller North Hall consists of 36 panels, 18 of which depict plants growing in the \
    British Isles. Both ceilings make extensive use of gilding for visual effect. Built of lath and plaster \
    to save costs, the ceilings are unusually fragile and require extensive maintenance and restoration."

In [8]:
test_sentences = nltk.sent_tokenize(test_article)

In [9]:
ner_by_spacy(test_sentences)

100%|██████████| 6/6 [00:00<00:00, 90.82it/s]


,Entities,Labels
0,the Natural History Museum,ORG
1,South Kensington,GPE
2,London,GPE
3,Alfred Waterhouse,PERSON
4,1881,DATE
5,Central Hall,FAC
6,162,CARDINAL
7,108,CARDINAL
8,the British Empire,GPE
9,54,DATE


In [10]:
ner_by_nltk(test_sentences)

100%|██████████| 6/6 [00:00<00:00, 30.54it/s]


,Entities,Labels
0,Natural History Museum,ORGANIZATION
1,South Kensington,GPE
2,London,GPE
3,Alfred Waterhouse,PERSON
4,Central Hall,ORGANIZATION
5,British,GPE
6,North Hall,LOCATION
7,British Isles,GPE


**Загрузка датафрейма с твитами**

In [11]:
import pickle

with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)

In [12]:
combine_df.head(5)

,id,label,tweet,tweet_token,tweet_token_f,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**NER с помощью spacy**

In [13]:
%%time

df_spacy = ner_by_spacy(combine_df['tweet'])

100%|██████████| 49159/49159 [05:22<00:00, 152.51it/s]

CPU times: user 5min 21s, sys: 2.41 s, total: 5min 23s
Wall time: 5min 22s


In [14]:
df_spacy.head(10)

,Entities,Labels
0,pdx,ORG
1,bihday,DATE
2,tomorrow,DATE
3,the next school year,DATE
4,the year,DATE
5,allin cavs,PERSON
6,cleveland,GPE
7,ireland,GPE
8,orlando standwithorlando,PERSON
9,today days,DATE


Топ-20 распознанных организаций, личностей, локаций, событий:

In [15]:
df_gb = df_spacy.groupby(['Entities', 'Labels'])[['Entities']].count()
df_gb.columns = ['counts']
df_gb.reset_index(inplace=True)
df_gb = df_gb[df_gb['Labels'].isin(['ORG', 'PERSON', 'GPE', 'FAC', 'EVENT'])]
df_gb.sort_values(by='counts', ascending=False).head(20)

,Entities,Labels,counts
408,america,GPE,225
8715,orlando,GPE,210
6710,london,GPE,182
12396,us,GPE,148
11180,tbt,ORG,119
862,aww,ORG,118
10085,selfie,PERSON,114
3660,florida,GPE,97
12324,uk,GPE,93
5327,japan,GPE,80


**NER с помощью nltk**

In [16]:
%%time

df_nltk = ner_by_nltk(combine_df['tweet'])

100%|██████████| 49159/49159 [02:37<00:00, 312.79it/s]

CPU times: user 2min 36s, sys: 1.77 s, total: 2min 37s
Wall time: 2min 37s


In [17]:
df_nltk.head(10)

,Entities,Labels
